## Access Analyzer

### ¿Qué es AWS IAM Access Analyzer?

AWS IAM Access Analyzer es un servicio de AWS que utiliza el razonamiento automatizado para analizar matemáticamente las políticas de control de acceso adjuntas a los recursos y determina a qué recursos se puede acceder públicamente o desde otras cuentas.

Access Analyzer también es capaz de validar tanto las políticas basadas en identidades como las basadas en recursos con más de 100 comprobaciones, cada una de ellas diseñada para mejorar su postura de seguridad y ayudarle a simplificar la administración de políticas a escala.

En este taller, utilizará estas dos características de Access Analyzer para validar las políticas de IAM en las plantillas de CloudFormation. Validar las políticas de IAM antes de implementarlas puede ayudar a evitar la implementación de recursos de AWS que permitan el acceso externo o políticas de IAM que no sigan las prácticas recomendadas.

Puede validar sus políticas mediante las comprobaciones de políticas de AWS IAM Access Analyzer. Access Analyzer valida su política con respecto a la gramática y las prácticas recomendadas de la política de IAM. Puede ver los resultados de la comprobación de validación de políticas que incluyen advertencias de seguridad, errores, advertencias generales y sugerencias para su política. Estos resultados proporcionan recomendaciones prácticas que le ayudarán a crear políticas funcionales y conformes a las mejores prácticas de seguridad.

In [42]:
import boto3
import json
from botocore.exceptions import ClientError
from pprint import pprint

# Create session
session = boto3.Session(profile_name='badbob')
client = session.client('accessanalyzer')

In [36]:
def check_response(response):
    if len(response['findings']) == 0:
        print('No errors have been found.')
    else:
        for find in response['findings']:
            pprint(find)

In [25]:
sample_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "ec2:RunInstance"
            ],
            "Resource": "arn:aws:ec2:*:*:instance/my-custom-instance"
        }
    ]
}

In [26]:
response = client.validate_policy(
    locale='EN',
    policyDocument=json.dumps(sample_policy),
    policyType='IDENTITY_POLICY'
)

check_response(response)

{'findingDetails': 'The action ec2:RunInstance does not exist.',
 'findingType': 'ERROR',
 'issueCode': 'INVALID_ACTION',
 'learnMoreLink': 'https://docs.aws.amazon.com/IAM/latest/UserGuide/access-analyzer-reference-policy-checks.html#access-analyzer-reference-policy-checks-error-invalid-action',
 'locations': [{'path': [{'value': 'Statement'},
                         {'index': 0},
                         {'value': 'Action'},
                         {'index': 0}],
                'span': {'end': {'column': 88, 'line': 1, 'offset': 88},
                         'start': {'column': 71, 'line': 1, 'offset': 71}}}]}


*Access Analyzer* ha encontrado un problema con la política de ejemplo: utilizaba la acción ec2:RunInstance que no existe. La salida del comando validate-policy incluye detalles sobre el hallazgo, así como la ubicación dentro del JSON de la política IAM donde se puede localizar el problema. Access Analyzer clasifica los hallazgos en cuatro tipos diferentes y el tipo **ERROR** es el más grave.

In [28]:
# Correct policy
sample_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "ec2:RunInstances"
            ],
            "Resource": "arn:aws:ec2:*:*:instance/my-custom-instance"
        }
    ]
}

response = client.validate_policy(
    locale='EN',
    policyDocument=json.dumps(sample_policy),
    policyType='IDENTITY_POLICY'
)

check_response(response)

No findings found.


### Validación de las políticas de control de servicios (SCP)

Los SCP tienen una sintaxis admitida [ligeramente diferente](https://docs.aws.amazon.com/organizations/latest/userguide/orgs_manage_policies_scps_syntax.html) y, por lo tanto, tienen diferentes comprobaciones de validación de políticas.

In [33]:
scp_sample = {
    "Version": "2012-10-17",
    "Statement": [{
        "Principal": "*",
        "Effect": "Deny",
        "Action": [
            "ec2:CreateInternetGateway"
        ],
        "Resource": "*"
    }]
}

In [34]:
response = client.validate_policy(
    locale='EN',
    policyDocument=json.dumps(scp_sample),
    policyType='SERVICE_CONTROL_POLICY'
)

check_response(response)

{'findingDetails': 'SCPs do not support specifying principals. Remove the '
                   'Principal or NotPrincipal element.',
 'findingType': 'ERROR',
 'issueCode': 'SCP_SYNTAX_ERROR_PRINCIPAL',
 'learnMoreLink': 'https://docs.aws.amazon.com/IAM/latest/UserGuide/access-analyzer-reference-policy-checks.html#access-analyzer-reference-policy-checks-error-scp-syntax-error-principal',
 'locations': [{'path': [{'value': 'Statement'},
                         {'index': 0},
                         {'value': 'Principal'}],
                'span': {'end': {'column': 57, 'line': 1, 'offset': 57},
                         'start': {'column': 54, 'line': 1, 'offset': 54}}}]}


In [37]:
# Correct scp policy
scp_sample = {
    "Version": "2012-10-17",
    "Statement": [{
        "Effect": "Deny",
        "Action": [
            "ec2:CreateInternetGateway"
        ],
        "Resource": "*"
    }]
}

response = client.validate_policy(
    locale='EN',
    policyDocument=json.dumps(scp_sample),
    policyType='SERVICE_CONTROL_POLICY'
)

check_response(response)

No errors have been found.


### Crear un analizador

In [51]:
try:
    response = client.create_analyzer(
        type='ACCOUNT',
        analyzerName='AccessAnalyzerCICDWorkshop'
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'ServiceQuotaExceededException':
        response = client.list_analyzers(
            type='ACCOUNT'
        )
        analyzer_name = response['analyzers'][0]['name']
        analyzer_arn = response['analyzers'][0]['arn']
        print(f'Using an existing analyser: {analyzer_name}')
except Exception as e:
    print(e)

Using an existing analyser: IAMAAEnable-X6txYUGu3Jlq


In [57]:
ACCOUNT_ID = session.client('sts').get_caller_identity().get('Account')

queue_policy = {
    "Version": "2012-10-17",
    "Id": "MyQueuePolicy",
    "Statement": [{
        "Sid":"AllowSendMessage",
        "Effect": "Allow",
        "Principal": {
            "AWS": "111122223333"
        },
        "Action": "sqs:SendMessage",
        "Resource": f"arn:aws:sqs:us-east-1:{ACCOUNT_ID}:MyQueue"
    }]
}

In [61]:
response = client.create_access_preview(
    analyzerArn=analyzer_arn,
    configurations={
        'string': {
            {
                'sqsQueue': {
                    'queuePolicy': json.dumps(queue_policy)
            }  
        }
    } 
    } 
)

TypeError: unhashable type: 'dict'